In [507]:
import yfinance as yf
import pandas as pd
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output, callback
import plotly.express as px
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc

lista_de_simbolos = ['MSFT34.SA', 'PETR3.SA', 'AAPL34.SA']

#definindo Periodos
PERIODOS = ['5y','2y','1y','9mo','6mo','3mo','1mo','5d','2d']

df_global = pd.DataFrame()


In [508]:

# #func para criar dataframe com todas as acoes nescessarias

def create_global_df(acoes:list, periodo:str='3mo' ):
    df_final = pd.DataFrame()
    dados = pd.DataFrame()
    for acao in acoes:
        dados = pd.DataFrame(yf.Ticker(acao).history(period=periodo)) #carrega o historico
        dados['Titulo'] = acao
        df_final = pd.concat([df_final,dados])
        
    return df_final.sort_values(by='Date')

df_global = create_global_df(lista_de_simbolos)
# df_global.head(3)

In [509]:
#desenhando desenvolvimento acao
def grafico_principal(dados):
    fig = px.line(data_frame=dados,x=dados.index ,y=['Open'], color='Titulo')
    return fig


In [510]:
df_global



,Open,High,Low,Close,Volume,Dividends,Stock Splits,Titulo
Date,,,,,,,,
2023-09-20 00:00:00-03:00,66.714966,66.714966,65.017387,65.017387,440655,0.0,0.0,MSFT34.SA
2023-09-20 00:00:00-03:00,36.071225,36.409150,35.839502,36.119499,8157200,0.0,0.0,PETR3.SA
2023-09-20 00:00:00-03:00,43.480356,43.520321,42.741028,42.771004,166511,0.0,0.0,AAPL34.SA
2023-09-21 00:00:00-03:00,65.426800,66.405409,65.057331,65.756332,108184,0.0,0.0,MSFT34.SA
2023-09-21 00:00:00-03:00,35.771916,36.235358,35.549851,35.598125,14641100,0.0,0.0,PETR3.SA
...,...,...,...,...,...,...,...,...
2023-12-19 00:00:00-03:00,37.959999,38.090000,37.520000,38.040001,10195200,0.0,0.0,PETR3.SA
2023-12-19 00:00:00-03:00,47.830002,47.939999,47.599998,47.750000,49148,0.0,0.0,AAPL34.SA
2023-12-20 00:00:00-03:00,75.540001,76.660004,75.349998,76.349998,131448,0.0,0.0,MSFT34.SA


# ALterando Cards de Noticias

In [511]:
@callback(
    Output(component_id='carousel-items', component_property='items'),
    Input(component_id='escolha-de-acoes', component_property='value'),
)
def carousel_update(acoes_selecionadas):
    n_item = 0
    list_items_carousel = []
    for ticket in acoes_selecionadas:
        for item in yf.Ticker(ticket).news:
            n_item += 1
            try:
                header = item['title']
                href = item['link']
                src = item['thumbnail']['resolutions'][0]['url']
            except :
                pass
            list_items_carousel.append({ 'Key': str(n_item),'external_link':True, 'header':header, 'href':href, 'src':src })
    return list_items_carousel

# Alterando Grafico principal

In [512]:
#add controle
@callback(
    Output(component_id='grafico-01', component_property='figure'),
    Input(component_id='escolha-de-acoes', component_property='value'),
    Input(component_id='escolha-periodo-acoes', component_property='value')

)
def update_graph_01(acoes_selecionadas, periodo = '3mo'):
    df_global = create_global_df(acoes_selecionadas,periodo)
    #fintrando dados
    dados_filtrados = df_global.where(df_global['Titulo'].isin(list(acoes_selecionadas)))
    return grafico_principal(dados_filtrados)
    
    

# Nao aceitar valores Nulos

In [513]:
@callback(
    Output(component_id='escolha-de-acoes', component_property='value'),
    Input(component_id='escolha-de-acoes', component_property='value')
)
def teste(acoes_selecionadas):
    if not len(acoes_selecionadas):
        return lista_de_simbolos[:1]
    return acoes_selecionadas


# Dash app

In [514]:
#inicializar app

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]
app = Dash(__name__, external_stylesheets=external_stylesheets)

#definindo layout app
app.layout = dbc.Container([
    dbc.Row([
    html.H1('Acompanhamento de Ações' ,style={'backgroundColor':'white','text-align': 'center'} ),
    ]),

    dbc.Row([
        dbc.Col([
            html.Div('Escolha a ação:',style={'backgroundColor':'white'}),
            dcc.Dropdown(id='escolha-de-acoes',placeholder="Escolha uma Ação", options=lista_de_simbolos, multi=True, value=lista_de_simbolos),

            
        ],width=10),
        dbc.Col([
            html.Div('Escolha Periodo:',style={'backgroundColor':'white'}),
            dcc.Dropdown(id='escolha-periodo-acoes',placeholder="Escolha uma Periodo", options=PERIODOS, value='3mo'),

            
        ],width=2)
    
    ]),

    # dbc.Row([
    # ]),

    dbc.Row([
    dcc.Graph(figure={}, id='grafico-01')
    ]),

    dbc.Row([
        dbc.Col(
            [
            html.Div('News:',style={'backgroundColor':'white','text-align': 'center'}),
            dbc.Carousel(id='carousel-items' , interval=1000,
            items=list_items_carousel,
            className="carousel-fade", style={'backgroundColor':'white','vertical-align': 'middle'}
            )
        ],
          width=6),

        dbc.Col([
            dash_table.DataTable(data=df_global[['Open','Close']].to_dict('records'), page_size=12, style_table={'overflowX': 'auto'})
        ], width=6),
    ]),

])



if __name__ == '__main__':
    app.run(debug=True)